In [1]:
import sagemaker
import os
import boto3

# import mlflow
import numpy as np
import json
import logging


from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import Join, JsonGet
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.parameters import ParameterString
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.steps import ProcessingStep
# from sagemaker.sklearn.processing import ScriptProcessor

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline import PipelineDefinitionConfig
from sagemaker import image_uris
from get_deployment_step import get_deployment_step

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [6]:
# # SageMaker Session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Logger yapılandırması
logger = logging.getLogger()
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [7]:
def get_lambda_step(
    project: str,
    bucket_name: str,
    process_instance_count_param: int,
    process_instance_type_param: str,
    evaluation_image_uri: str,
    region: str,
    
    lambda_check_function_arn: str,
    model_package_arn: str,
    step_suffix: str
):

    sagemaker_session = sagemaker.Session()
    role = sagemaker.get_execution_role()

    logger.info("Starting get_lambda_step")

    func = Lambda(function_arn=lambda_check_function_arn)

    # Lambda Step: Model approval status sorgulama
    return LambdaStep(
        name=f"CheckApprovalStatus{step_suffix}",
        lambda_func=func,
        inputs={"model_package_arn": model_package_arn},
        outputs=[
            LambdaOutput(output_name='Payload')
        ]
    )

In [8]:
def get_manuel_approve_step(
    project: str,
    bucket_name: str,
    process_instance_count_param: int,
    process_instance_type_param: str,
    evaluation_image_uri: str,
    region: str,

    model_path: str,
    check_status_step: LambdaStep,
    deployment_step: ProcessingStep,
    lambda_wait_function_arn: str,
    model_package_arn: str,
    wait_step_suffix: str,
    condition_step_suffix: str
):
    logger.info("Starting manuel_approve_step")

    wait_step = LambdaStep(
        name=f"WaitForApproval{wait_step_suffix}",
        lambda_func=Lambda(
            function_arn=lambda_wait_function_arn
        ),
        inputs={"model_package_arn": model_package_arn},
        outputs=[
            LambdaOutput(output_name='Payload')
        ]
    )

    # Condition Step: Model approval status kontrol etme
    return ConditionStep(
        name=f"CheckIfApproved{condition_step_suffix}",
        conditions=[
            ConditionEquals(
                left=JsonGet(
                    step=check_status_step,
                    # property_file="Payload",
                    s3_uri=Join(
                            on="/",
                            values=[
                                f"s3://wildfires",
                                "model_status",
                                f"{model_package_arn}.json"
                            ],),
                    json_path="$.ApprovalStatus"
                ),
                right="Approved"
            )
        ],
        if_steps=[
            deployment_step
            # ModelStep(
            #     name="DeployModel",
            #     model=PyTorchModel(
            #         model_data=f"s3://{bucket_name}/{model_path}",
            #         role=role,
            #         framework_version='2.0.0',
            #         py_version='py310',
            #         entry_point='inference.py'
            #     ),
            #     instance_count=1,
            #     instance_type="ml.g5.xlarge",
            # )
        ],
        else_steps=[
            wait_step,
            check_status_step
        ]
    )


In [9]:
def get_parameters() -> dict:
    process_instance_count_param = ParameterInteger(
        name="ProcessingInstanceCount",
        default_value=1
    )
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.large",
    )
    #------------------------------------------------

    #------------------------------------------------

    return {
        "process_instance_count_param": process_instance_count_param,
        "process_instance_type_param": process_instance_type_param,
    }

In [10]:
def get_pipeline(
    session: sagemaker.Session,
    parameters: dict,
    constants: dict,
    sklearn_image_uri: str
):
    pipeline_def_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

    # processor_step = get_processor_step(
    #     project=constants["project"],
    #     bucket_name=constants["bucket_name"],
    #     process_instance_count_param=parameters["process_instance_count_param"],
    #     process_instance_type_param=parameters["process_instance_type_param"],
    #     sklearn_image_uri=sklearn_image_uri,
    #     region=constants["region"],
    # )

    # # ------------------------------------------------
    # evaluation_step = get_evaluation_step(
    #     project=constants["project"],
    #     bucket_name=constants["bucket_name"],
    #     process_instance_count_param=parameters["process_instance_count_param"],
    #     process_instance_type_param=parameters["process_instance_type_param"],
    #     evaluation_image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
    #     region=constants["region"],
        
    #     test_metadata_prefix='sagemaker/fire-image-classification',
    #     best_model_prefix='models',
    #     test_metadata_file='test.pkl',
    #     best_model_file='model_resnet18.tar.gz',
    #     result_prefix='evaluation/dummy',
    #     data_dir='s3://wildfires/sagemaker/fire-image-classification',
    #     model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
    # )

    # ------------------------------------------------

    # ------------------------------------------------

    suffix = "Initial"

    check_status_step = get_lambda_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
        region=constants["region"],

        lambda_check_function_arn='arn:aws:lambda:eu-central-1:567821811420:function:LambdaWildfireCheckStatus',
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1',
        step_suffix=suffix
    )

    deployment_step = get_deployment_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
        region=constants["region"],
        
        # test_metadata_prefix='sagemaker/fire-image-classification',
        # best_model_prefix='models',
        # test_metadata_file='test.pkl',
        # best_model_file='model_resnet18.tar.gz',
        # result_prefix='evaluation/dummy',
        # data_dir='s3://wildfires/sagemaker/fire-image-classification',
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
    )

    conditional_step = get_manuel_approve_step(
        project=constants["project"],
        bucket_name=constants["bucket_name"],
        process_instance_count_param=parameters["process_instance_count_param"],
        process_instance_type_param=parameters["process_instance_type_param"],
        evaluation_image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
        region=constants["region"],
        
        model_path='',
        check_status_step=check_status_step,
        deployment_step=deployment_step,
        lambda_wait_function_arn='arn:aws:lambda:eu-central-1:567821811420:function:LambdaWildfireWaitForApproval',
        model_package_arn='arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1',
        wait_step_suffix=suffix,
        condition_step_suffix=suffix
    )

    # ------------------------------------------------

    return Pipeline(
        name=f"{constants['project']}-pipeline",
        parameters=[parameters[key] for key in parameters],
        pipeline_definition_config=pipeline_def_config,
        steps=[
            # processor_step,
            # evaluation_step,
            conditional_step
            # check_status_step
        ],
    )

In [11]:
print(os.getcwd())

parameters = get_parameters()

constants = {
    "region": "eu-central-1",
    "project": "wildfire-dumm-8",
    "bucket_name": "wildfires",
    "sklearn_image_uri_version": "1.2-1",
}

session = sagemaker.Session(boto3.Session(region_name=constants["region"]))

sklearn_image_uri = image_uris.retrieve(
    framework="sklearn",
    region=constants["region"],
    version=constants["sklearn_image_uri_version"],
)

pipeline = get_pipeline(
    session=session,
    parameters=parameters,
    constants=constants,
    sklearn_image_uri=sklearn_image_uri,
)

pipeline.upsert(role_arn=sagemaker.get_execution_role())

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
2024-06-27 23:00:27,525 - sagemaker.image_uris - INFO - Defaulting to only available Python version: py3


/home/sagemaker-user/Steps/Job 5 Manuel App and Deployment


INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
2024-06-27 23:00:27,558 - sagemaker.image_uris - INFO - Defaulting to only supported image scope: cpu.
INFO:root:Starting get_lambda_step
2024-06-27 23:00:27,898 - root - INFO - Starting get_lambda_step
INFO:root:Starting manuel_approve_step
2024-06-27 23:00:28,025 - root - INFO - Starting manuel_approve_step
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/lambda_step.py:165: UserWarning: Lambda function won't be updated because zipped_code_dir                 or script is not provided.
  warnings.warn(


{'PipelineArn': 'arn:aws:sagemaker:eu-central-1:567821811420:pipeline/wildfire-dumm-8-pipeline',
 'ResponseMetadata': {'RequestId': 'b8030a4b-ca1a-43ea-91c3-fbbb21a2c1fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b8030a4b-ca1a-43ea-91c3-fbbb21a2c1fb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Thu, 27 Jun 2024 23:00:28 GMT'},
  'RetryAttempts': 0}}

In [8]:
s3 = boto3.client('s3')
# s3://wildfires/model_status/arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1.json
prefix = 'model_status/arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049'
s3.download_file('wildfires', 'model_status/arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1.json', '1_.json')

In [11]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.lambda_step import LambdaStep
from sagemaker.lambda_helper import Lambda
import boto3

# Initialize boto3 client
sagemaker_client = boto3.client('sagemaker')
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

# Lambda function ARN
lambda_function_arn = 'arn:aws:lambda:eu-central-1:567821811420:function:LambdaWildfireCheckStatus'

# Lambda function
lambda_function = Lambda(function_arn=lambda_function_arn)

# Lambda step
lambda_step = LambdaStep(
    name="LambdaCheckStatusStep",
    lambda_func=lambda_function,
    inputs={
        'model_package_arn': 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
    }
)

# Define the pipeline
pipeline = Pipeline(
    name="SageMakerLambdaPipeline-11",
    steps=[lambda_step],
    sagemaker_session=sagemaker.Session()
)

# Create the pipeline
pipeline.create(role_arn=role)

# Execute the pipeline
execution = pipeline.start()

ClientError: An error occurred (ValidationException) when calling the CreatePipeline operation: Pipeline names must be unique within an AWS account and region. Pipeline with name (SageMakerLambdaPipeline-11) already exists.

In [2]:
boto_session = boto3.Session()
region = boto_session.region_name

print(region)
# print(args.region)

sm = boto3.client('sagemaker', region_name='eu-central-1')

s3 = boto3.client('s3')
print('Deployment job is started')

BUCKET_NAME = 'wildfires'
model_package_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'

s3.download_file('wildfires', f'model_status/{model_package_arn}.json', 'status_2.json')

eu-central-1
Deployment job is started


In [45]:
client = boto3.client('sagemaker')
model_package_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'
s3 = boto3.client('s3')
BUCKET_NAME = 'wildfires'

response = client.describe_model_package(ModelPackageName=model_package_arn)
response['ModelApprovalStatus']

'PendingManualApproval'

In [1]:
%pwd

'/home/sagemaker-user/Steps/Job 5 Manuel App and Deployment'

In [ ]:
s3 = boto3.client('s3')

BUCKET_NAME = 'wildfires'

s3.upload_file(
    f'manuel_check.py',
    BUCKET_NAME,
    f"scripts/manuel_check.py",
)

In [ ]:
role = get_execution_role()
sm_session = sagemaker.Session()

boto_session = boto3.Session()
region = boto_session.region_name

constants = {
    "bucket_name": "wildfires",
    # "sklearn_image_uri_version": "1.2-1",
    "region": boto_session.region_name
}

script_path = sm_session.upload_data(
    path='manuel_check.py',
    bucket=constants["bucket_name"],
    key_prefix="scripts"
)

# test_metadata_path = 's3://wildfires/sagemaker/fire-image-classification'

# best_model_path = 's3://fire-project-hs/34/36ba5f52b9944395ab2ccf94aed56e5a/artifacts/models'

model_package_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'

# sklearn_image_uri = image_uris.retrieve(
#         framework="sklearn",
#         region=constants["region"],
#         version=constants["sklearn_image_uri_version"],
#     )

script_processor = ScriptProcessor(
    role=role,
    image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
    command=['python3'],
    instance_count=1,
    instance_type='ml.m5.large'
)

script_processor.run(
    job_name='wildfire-manuel-check-job-t01',
    code=script_path,
    arguments=[
        '--test_metadata_bucket', constants["bucket_name"],
        '--best_model_bucket', constants["bucket_name"],
        '--test_metadata_prefix', 'sagemaker/fire-image-classification',
        '--best_model_prefix', 'models',
        '--test_metadata_file', 'test.pkl',
        '--best_model_file', 'model_resnet18.tar.gz',
        '--result_bucket', constants["bucket_name"],
        '--result_prefix', 'evaluation/dummy',
        '--result_file', 'eva-result-',
        '--data_dir', 's3://wildfires/sagemaker/fire-image-classification',
        '--model_package_arn', model_package_arn,
        '--region', constants["region"]
    ]
)